## Interfaces and implementations lab

In this lab, you will define an environment interface, which you will then proceed to use as a base for extending our calculator from last time with variables.

We reproduce here (and simplify) some code from last time, that will enable us to create an expression tree...

In [1]:
def typer(token):
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return str(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token

In [2]:
def parse(loc):
    return syn(lex(loc))

In [3]:
parse('(+ 1 2.0 3 4 (* 34.1 2))')

['+', 1, 2.0, 3, 4, ['*', 34.1, 2]]

Notice that instead of using our BinaryTree class from last time, we are simply using embedded lists..

We'll assign variables using the `let` form..

In [4]:
parse('(let a 5.5)')

['let', 'a', 5.5]

### Informal Spec for the calculator

Let us write down an informal spec for our calculator language

- it will work in in-fix notation
- we support all kinds of numbers
- it only supports unarybinary operations from the math module (see below)
- we also support +/-/truediv/x/max/min/round
- there is no notion of booleans or None yet (although we will see a None leak in)
- we support variable assignment with `(let a (+ 1 2))` for example
- there is only one environment in which bindings are made
- while we have numbers coming in, all our output is in python, and is only reflected in our language via `let`

In [ ]:
import math
vars(math)

### Q1.

Here is an Environment ABC which defines the following abstract methods 

In [7]:
import abc
class Environment(abc.ABC):
    """
    This is the interface for an Environment. The client for 
    this interface is a language intepreter. 
    """
    @classmethod
    @abc.abstractmethod
    def empty(cls):
        return cls()
    
    @abc.abstractmethod
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
    
    @abc.abstractmethod
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        
    @abc.abstractmethod
    def lookup(variable):
        """
        return the unique binding of the variable in the current 
        environment. If it is not found raise a NameError as below
        """
        raise NameError("{} not found in Environment".format(vaiable))

Here is the Linked list class from a few labs back. We'll use a linked list as a repository for bindings. We will consider an implementation where newer bindings for existing variables occur closer to the head in the linked list. Notice that we added a `repOK` which is an identity function. Take care to see where it is used.

In [8]:
import reprlib, numbers


class LL:
    """
    >>> A = LL()  
    >>> A[0]
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
    >>> A.insert_front(1)
    >>> A[0]
    1
    >>> A.insert_back(2)
    >>> A[1]
    2
    >>> A
    LL([1,...])
    >>> myll = LL.from_components([1,2])
    >>> myll[1]
    1
    >>> len(myll)
    2
    >>> myll[2]
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
    >>> myll[0:1]
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
    """
    @classmethod
    def from_components(cls, components):
        inst = cls(components[0])
        for c in components[1:]:
            inst.insert_front(c)
        return inst
        
    def repOK(self, element):
        return element
    
    def __init__(self, head=None):
        if head is None:
            self._headNode = None
        else:
            head = self.repOK(head)
            self._headNode = [head, None]
            
    def insert_front(self, element):
        element = self.repOK(element)
        new_node = [element, None]
        new_node[1] = self._headNode
        self._headNode = new_node
        
    def insert_back(self, element):
        element = self.repOK(element)
        new_node = [element, None]
        curr_ptr = self._headNode
        while curr_ptr[1] is not None:
            curr_ptr = curr_ptr[1]
        curr_ptr[1]= new_node
        
    def __repr__(self):
        class_name = type(self).__name__
        if len(self)==0:
            components=""
        else:
            components = reprlib.repr(self[0])
        return '{}([{},...])'.format(class_name,components)


    def __len__(self):
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            return 0
        while 1:
            count = count + 1
            if curr_ptr[1] is None:
                break
            curr_ptr = curr_ptr[1]
        return count    
    
    def __getitem__(self, index):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    return curr_ptr[0]
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))
         
    def index(self, element):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            msg = 'trying to get index from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while 1:
            if curr_ptr[0] == element:
                return count
            if curr_ptr[1] is None:
                msg = '{element} is not in {class_name}' 
                raise ValueError(msg.format(element=element, class_name=class_name))
            count += 1
            curr_ptr = curr_ptr[1]
            
    def remove(self, element):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        prev_ptr = None
        if curr_ptr==None:
            msg = 'remove from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while 1:
            if curr_ptr[0] == element:
                if prev_ptr is None:
                    self._headNode = curr_ptr[1]
                else:
                    prev_ptr[1] = curr_ptr[1]
                return None
            if curr_ptr[1] is None:
                msg = '{element} is not in {class_name}' 
                raise ValueError(msg.format(element=element, class_name=class_name))
            prev_ptr=curr_ptr
            curr_ptr = curr_ptr[1]
        
    def remove_front(self):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        if curr_ptr==None:
            msg = 'remove from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        self._headNode = curr_ptr[1]
        return curr_ptr[0]
    

### Q1.

Inherit from class `LL` to create a TupleLL where the values in the linked list are tuples of the type `(key, value)`. After inheriting, add an additional method `first_match` which returns a tuple `(index_of_nearest_element, nearest element)` to the head where the first member of the tuple matches `key`. Also add another method `all_matches` which returns a list of tuples of allmatches, sorted by distance to head. Currently, the structure of the other methods in the linked list do not need to know about this 2-element tuple restriction on the elements. 

In [167]:
class TupleLL(LL):
    """
    A linked list whose elements must be tuples.
    
    RepInv: any insertion should leave a list with only tuples as members.
    """
    # Could be implemented using a check on the whole list
    def repOK(self, element):
        assert isinstance(element, tuple), "element needs to be a tuple"
        return element
    
    def first_match(self, key):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            msg = 'trying to get first match from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while True:
            if curr_ptr[0][0] == key:
                return count, curr_ptr[0][1]
            if curr_ptr[1] is None:
                msg = '{element} is not in {class_name}' 
                raise ValueError(msg.format(element=key, class_name=class_name))
            count += 1
            curr_ptr = curr_ptr[1]
            
    def all_matches(self, key):
        class_name = type(self).__name__
        curr_ptr = self._headNode
        count = 0
        matches = list()
        if curr_ptr == None:
            msg = 'trying to get all matches from empty {class_name}' 
            raise IndexError(msg.format(class_name=class_name))
        while True:
            if (curr_ptr[0][0] == key):
                matches.append(curr_ptr[0])
            if (curr_ptr[1] is None):
                return matches
            count += 1
            curr_ptr = curr_ptr[1]

In [168]:
l = TupleLL()
l.insert_front(('a', 1))

In [169]:
l

TupleLL([('a', 1),...])

In [170]:
l.insert_front((3,))

### Q2.

Implement a inplementation class `Env1` that impements the environment interface using the `TupleLL`. Let us ask what the AbsFun and RepInv are for this implementation. Write an AbsFun for it, noting that in this concrete representation, there can be multiple key-value pairs in the environment for the same key, and that there is a way to disambiguate the correct one. The Absfun should limit the keys and values allowed for our bindings as well. Where should these checks be made..thats a big question!

We will not make any checks here that in bindings the keys are strings and values are legitimate in our language.

In [48]:
class Env1:
    """
    AbsFun: The list [[a, [b, ..., z]]]]] represents the linked list a->b->...->z, where each entry is a key-value
    tuple. There may be multiple entries with the same key. If there are two entries with the same key, whichever
    comes first is the valid one.
    [None] represents the empty linked list.
    """
            
    def __init__(self, head=None):
        self._storage = TupleLL(head)
        
    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
        self._storage.insert_front((variable, value))
    
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        for k, v in envdict.items():
            self._storage.insert_front((k, v))
        
    def lookup(self, variable):
        """
        return the unique binding of the variable in the current 
        environment. If it is not found raise a NameError as below
        """
        try:
            return self.first_match(variable)
        except ValueError or IndexError:
            raise NameError("{} not found in Environment".format(vaiable))

### Q2b.

Now let us write and test a RepInv for this implementation. Note that just like in the case with the list-with-duplicates implementation of a set, its hard to come up with any uniqueness-of-binding RepInv. But we can ask the following: a newer binding overrides an old one. Write a Repinv which implements this and include a repOK methodwhich makes sure that any methods mutating the environment respect this RepInv and any observers return the correct binding. Hint: the `repOK` will need a signature `def repOK(self, key, value)` and also be used in the `lookup` method. It will use `all_matches`, and not be returning anything.

Note that we might want to check that keys are strings and values are legitimate values in our language, but we wont do it for now. We'll come back to it in a few days.

In [171]:
class Env1:
    """
    AbsFun: The list [[a, [b, ..., z]]]]] represents the linked list a->b->...->z, where each entry is a key-value
    tuple. There may be multiple entries with the same key. If there are two entries with the same key, whichever
    comes first is the valid one.
    [None] represents the empty linked list.
    The list element [a, [1, ...]] represents the mapping from the variable a to the number 1.
    
    RepInv: A newer binding overrides an old one.
    """
            
    def __init__(self, head=None):
        self._storage = TupleLL(head)
        
    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
        self._storage.insert_front((variable, value))
    
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        for k, v in envdict.items():
            self._storage.insert_front((k, v))
        
    def lookup(self, variable):
        """
        return the unique binding of the variable in the current 
        environment. If it is not found raise a NameError as below
        """
        try:
            return self._storage.first_match(variable)
        except ValueError or IndexError:
            raise NameError("{} not found in Environment".format(variable))
            
    def repOk(self, key, value):
        assert self._storage.first_match(key)[1] == value

Lets use this class in a function which creates a global environment for our calculator referenced as `globenv`. we first register this implementation for the `Environment` interface

In [172]:
Environment.register(Env1)
def one_env(envclass):
    "An environment with some Scheme standard procedures."
    import math, operator as op
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
    })
    return env

globenv = one_env(Env1)

### Q3.

Write a function, `def eval_ptree(x, env=globenv):` which evaluates the parse tree for a given parsed expressionlist, x. This function works recursively, in postorder mode, to traverse the expression tree. We have implemented part of it for you. You need to complete the current `elif` with a post-order traversal to "extend" the environment(one liner). Finally there is an `else` clause to implement which looks up the operator/function in the environment, uses post-order traversal to calculate the arguments, and then runs the operator on the arguments....

In [198]:
def eval_ptree(x, env=globenv):
    if isinstance(x, str):      # variable lookup
        return env.lookup(x)[1]
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None 
    elif x[0] == 'let':         # variable definition
        (let, var, expression) = x
        #postorder traversal by nested eval is needed below
        env.extend(var, eval_ptree(expression))
    else: # operator thing
        operation = globenv.lookup(x[0])[1]
        return operation(*[eval_ptree(i) for i in x[1:]])

In [199]:
eval_ptree(parse('(* 1 5)'))

5

Here is a simple check. (in addition to any tests you might have written)

In [200]:
eval_ptree(parse('(let a ())')) 
eval_ptree(parse('(+ a 1)')) 

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [201]:
eval_ptree(parse('(sin 3)'))

0.1411200080598672

This is a very unsophisticated language. Nonsense like the above is allowed...we dont have a concept of None as yet..we'll fix it later...

In [202]:
eval_ptree(parse('(let a 5.5)'))
eval_ptree(parse('(* (+ a 0.5) (+ a 4))')) #should give 57

57.0

We write a `Program` class which takes a newline separated program, and can show it, parse it, and run it.

In [203]:
class Program():
    
    def __init__(self, program, env):
        self.program = [e.strip() for e in program.split('\n')]
        self.env = env
        
    def __iter__(self):
        for line in self.program:
            yield line
    
    def parse(self):
        for l in iter(self):
            yield parse(l)
            
    def run(self):
        for l in iter(self):
            yield eval_ptree(parse(l), self.env)

In [204]:
program = """
(let radius 5)
(* pi (* radius radius))
"""

In [205]:
p=Program(program, globenv)
list(iter(p))

['', '(let radius 5)', '(* pi (* radius radius))', '']

In [206]:
for s in p.parse():
    print(s)

[]
['let', 'radius', 5]
['*', 'pi', ['*', 'radius', 'radius']]
[]


In [207]:
for result in p.run():
    print(result)

None
None
78.53981633974483
None


### Q4.

Implement another version of environment, `Env2`, which used dictionaries under the hood. Write an AbsFun and RepInv for it. In general we ought to check RepInv's, but do we even need to do it in this case? Register it

In [213]:
class Env2:
    """
    AbsFun: The dict {'a':i_1, ..., 'z':i_n} represents the set of mappings from the variables a, ..., z to the
    variables i_1, ..., i_n. There may not be multiple entries with the same key.
    {} represents the empty dict.
    
    RepInv: A newer binding overrides an old one.
    """
            
    def __init__(self, head=None):
        self._storage = {}
        
    @classmethod
    def empty(cls):
        return cls()
    
    def extend(self, variable, value):
        """
        extend an existing environment by binding variable to value.
        The values must be an acceptable value in the language. If the
        same variable is used twice the newer value must be bound.
        """
        self._storage[variable] = value
    
    def extend_many(self, envdict):
        """
        extend the current environment by values in the dictionary
        envdict. If the dictionary contains variables already in the
        environment, the newer values from the dictionary are bound
        """
        for k, v in envdict.items():
            self._storage[k] = v
        
    def lookup(self, variable):
        """
        return the unique binding of the variable in the current 
        environment. If it is not found raise a NameError as below
        """
        try:
            return self._storage[variable]
        except KeyError:
            raise NameError("{} not found in Environment".format(variable))
            
    def repOk(self, key, value):
        assert self._storage[key] == value # trivial

Let us test it out in the wild....

In [214]:
globenv2 = one_env(Env2)

In [215]:
p2=Program(program, globenv2)
for result in p2.run():
    print(result)

None
None
78.53981633974483
None


### Adding a repl

In [216]:
def repl(prompt='calc> '):
    while True:
        try:
            val = eval_ptree(parse(input(prompt)), globenv2)
        except (KeyboardInterrupt, EOFError):
            break
        if val is not None: 
            print(val)

In [ ]:
repl()# to get out of the repl in the notebook just cause an exception like below

Notice we have not tamped down out language ny formally speifying what values are allowed in it. At the very least we should formalize variables as strings and constants as numbers.Real. Whats the right place to do this? The parser? The evaluater? We will come to all of this soon.